In [32]:
pip install openpyxl

   ---------------------------------------- 0.0/251.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/251.3 kB ? eta -:--:--
   --------- ----------------------------- 61.4/251.3 kB 812.7 kB/s eta 0:00:01
   ----------------- -------------------- 112.6/251.3 kB 930.9 kB/s eta 0:00:01
   ----------------- -------------------- 112.6/251.3 kB 930.9 kB/s eta 0:00:01
   ----------------------------------- ---- 225.3/251.3 kB 1.1 MB/s eta 0:00:01
   ----------------------------------- ---- 225.3/251.3 kB 1.1 MB/s eta 0:00:01
   ----------------------------------- ---- 225.3/251.3 kB 1.1 MB/s eta 0:00:01
   -------------------------------------  245.8/251.3 kB 718.0 kB/s eta 0:00:01
   -------------------------------------- 251.3/251.3 kB 669.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import openai
import requests
import PyPDF2
import re
import os
import requests
import pandas as pd
import tiktoken
import time
from openai import OpenAI
from io import StringIO
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
import numpy as np
import ast

In [2]:
client = OpenAI(
     api_key=os.environ.get("OPENAI_API_KEY"),)

In [3]:
#extract paper titles from a list of file paths
def extract_titles(file_paths):
    
    titles = []
    for path in file_paths:
        # Extract the file name without the extension
        match = re.search(r'([^/\\]+)\.pdf$', path)
        if match:
            title = match.group(1)
            # Remove the specified suffix if present
            titles.append(title)
    
    return pd.DataFrame(titles, columns=['Paper_title'])

In [4]:
#define a function to extract table titles
def table_titles(pdf_path):
    def extract_table_titles(pdf_path):
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            num_pages = len(reader.pages)
            table_titles = []
    
            # Regular expression to find table titles
            table_title_regex = re.compile(r'[Tt][Aa][Bb][Ll][Ee]+\.?+\s*[A-Z]?\d+\.?\s+[A-Z]+.*')
    
            for i in range(num_pages):
                page = reader.pages[i]
                text = page.extract_text()
                if text:
                    found_titles = table_title_regex.findall(text)
                    table_titles.extend(found_titles)
            return table_titles
            
    table_titles = extract_table_titles(pdf_path)
    single_cell_content = ' | '.join(table_titles)
    df = pd.DataFrame([[single_cell_content]], columns=['Table_title'])
    df['Table_title'] = df['Table_title'].str.replace(r'\n', ' ', regex=True)
                
    return df

In [5]:
def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [6]:
def df_to_csv(df, file_name):
    """Write a DataFrame to a CSV file."""
    df.to_csv(file_name, index=False, escapechar='\\')

In [13]:
#read all pdf files from a specific directory
directory = 'C:/Users/89751/OneDrive/desktop/Text-embedding classification/Document/'
files = os.listdir(directory)
pdf_files = [os.path.join(directory, file) for file in files]  #
pdf_files

['C:/Users/89751/OneDrive/desktop/Text-embedding classification/Document/49.pdf',
 'C:/Users/89751/OneDrive/desktop/Text-embedding classification/Document/50.pdf']

In [14]:
df1 = extract_titles(pdf_files)
df1

,Paper_title
0,49
1,50


In [15]:
df2 = [table_titles(path) for path in pdf_files]

# Concatenate all DataFrames into a single DataFrame
df2 = pd.concat(df2, ignore_index=True)

df = pd.concat([df1, df2], axis=1)
df

,Paper_title,Table_title
0,49,Table 1. LCA of bio-ethanol produced from conv...
1,50,Table 1 Input and output ﬂows of WS-EtOH produ...


In [16]:
print(df['Table_title'][0])

Table 1. LCA of bio-ethanol produced from conventional food  | Table 1  Ethanol pathways ’ LCA results compared in different studies.   | Table 2. These  | Table 4.  Energy-efficient scenario: the current LDG production process, as the  | Table 2  Data inventory for LDG-ethanol process.   | Table 5. LDG-ethanol route has  | Table 3  Data inventory for Corn-ethanol and Coal-ethanol.   | Table 4  Scenario description.   | Table 5  LCA characterization results.  


In [17]:
df['Table_title'] = df['Table_title'].str.split('|')

# Exploding the 'Table_title' list into multiple rows
df_split = df.explode('Table_title')
df_split.reset_index(drop=True, inplace=True)

df_split

,Paper_title,Table_title
0,49,Table 1. LCA of bio-ethanol produced from conv...
1,49,Table 1 Ethanol pathways ’ LCA results compa...
2,49,Table 2. These
3,49,Table 4. Energy-efficient scenario: the curr...
4,49,Table 2 Data inventory for LDG-ethanol proce...
5,49,Table 5. LDG-ethanol route has
6,49,Table 3 Data inventory for Corn-ethanol and ...
7,49,Table 4 Scenario description.
8,49,Table 5 LCA characterization results.
9,50,Table 1 Input and output ﬂows of WS-EtOH produ...


In [19]:
file_name = 'C:/Users/89751/OneDrive/Desktop/Output_test.xlsx'
df_split.to_excel(file_name)